In [1]:
# link = "https://fifemon.fnal.gov/kibana/app/kibana#/visualize/edit/33d02c40-8b41-11ee-804b-5759672b811c?_g=(refreshInterval:(pause:!t,value:0),time:(from:'2022-01-01T06:00:00.000Z',mode:absolute,to:'2023-11-25T03:09:25.714Z'))&_a=(filters:!(),linked:!f,query:(language:lucene,query:'Jobsub_Group:dune'),uiState:(vis:(params:(sort:(columnIndex:!n,direction:!n)))),vis:(aggs:!((enabled:!t,id:'1',params:(field:SlotHours),schema:metric,type:sum),(enabled:!t,id:'3',params:(field:MachineAttrGLIDEIN_DUNESite0,missingBucket:!f,missingBucketLabel:Missing,order:desc,orderBy:'1',otherBucket:!f,otherBucketLabel:Other,size:50),schema:bucket,type:terms),(enabled:!t,id:'5',params:(customInterval:'2h',drop_partials:!f,extended_bounds:(),field:'@timestamp',interval:M,min_doc_count:1,timeRange:(from:'2022-01-01T06:00:00.000Z',mode:absolute,to:'2023-11-25T03:09:25.714Z'),useNormalizedEsInterval:!t),schema:bucket,type:date_histogram),(enabled:!t,id:'4',params:(filters:!((input:(query:'NOT(Owner:dunepro)%20AND%20NOT(Jobsub_SubGroup:mars)'),label:Analysis),(input:(query:'Owner:dunepro'),label:Production),(input:(query:'Jobsub_SubGroup:mars'),label:MARS),(input:(query:'*'),label:Total))),schema:bucket,type:filters)),params:(perPage:24,showMetricsAtAllLevels:!f,showPartialRows:!f,showTotal:!f,sort:(columnIndex:!n,direction:!n),totalFunc:sum),title:'DUNE%20monthly%20slot%20hours%20by%20site%20and%20role',type:table))"

In [2]:
# go to that link, go to inspect and save as formatted csv


In [3]:
# define  bydate(array=None,types=None,locations=None,dates=None,units=None,tag=None):

In [4]:
 def bydate(array=None,types=None,locations=None,dates=None,units=None,format="10.3f",tag=None):
    # metdefhod that makes a table from an array indexed by type, location and date by location and date.
    # tag is a tag that tells what the array was
    # adds a sum across both row and column
    # output is a csv file of the table
    lowdate = dates[0]
    highdate = dates[-1]
    print (lowdate,highdate)
    header = "   time in %s by %s     "%(units,tag)
    header2 = header

    for date in dates:
        header += "%10s"%date
        header2 += ",%s"%date
    header += "     Total"
    header2 += ",Total\n"
    form = ", %%%s"%format
    out = {}
    for type in types:
        outname = "output/%s_%s_%s_%s_%s.csv"%(type,tag,units,lowdate,highdate)
        out[type] = open(outname,'w')
        out[type].write(header2)

    print (locations) 

    for type in types:  
        totalbydate = {}
        for date in dates:
            totalbydate[date] = 0.0
        totaltotal = 0.0
        for site in locations:
            result = "%30s"%site
            outstring = "%s"%site
            total = 0.0
            
            #print (form)
            for date in dates:
                result += " %10.3f"%(array[type][site][date])
                outstring += form%(array[type][site][date])
                total += (array[type][site][date])
                totalbydate[date]+= (array[type][site][date])
            totaltotal+=total           
            outstring += form%total
            outstring += "\n"
            #print (outstring)
            out[type].write(outstring)
        outstring = "%s"%"Total"
        for date in dates:
            outstring += form%(totalbydate[date])
        outstring += form%totaltotal 
        outstring += "\n"
        out[type].write(outstring)
        out[type].close()

In [5]:
import csv

# ken
#  "https://fifemon.fnal.gov/kibana/app/kibana#/visualize/edit/33d02c40-8b41-11ee-804b-5759672b811c?_g=(refreshInterval:(pause:!t,value:0),time:(from:'2022-01-01T06:00:00.000Z',mode:absolute,to:'2023-11-25T03:09:25.714Z'))&_a=(filters:!(),linked:!f,query:(language:lucene,query:'Jobsub_Group:dune'),uiState:(vis:(params:(sort:(columnIndex:!n,direction:!n)))),vis:(aggs:!((enabled:!t,id:'1',params:(field:SlotHours),schema:metric,type:sum),(enabled:!t,id:'3',params:(field:MachineAttrGLIDEIN_DUNESite0,missingBucket:!f,missingBucketLabel:Missing,order:desc,orderBy:'1',otherBucket:!f,otherBucketLabel:Other,size:50),schema:bucket,type:terms),(enabled:!t,id:'5',params:(customInterval:'2h',drop_partials:!f,extended_bounds:(),field:'@timestamp',interval:M,min_doc_count:1,timeRange:(from:'2022-01-01T06:00:00.000Z',mode:absolute,to:'2023-11-25T03:09:25.714Z'),useNormalizedEsInterval:!t),schema:bucket,type:date_histogram),(enabled:!t,id:'4',params:(filters:!((input:(query:'NOT(Owner:dunepro)%20AND%20NOT(Jobsub_SubGroup:mars)'),label:Analysis),(input:(query:'Owner:dunepro'),label:Production),(input:(query:'Jobsub_SubGroup:mars'),label:MARS),(input:(query:'*'),label:Total))),schema:bucket,type:filters)),params:(perPage:24,showMetricsAtAllLevels:!f,showPartialRows:!f,showTotal:!f,sort:(columnIndex:!n,direction:!n),totalFunc:sum),title:'DUNE%20monthly%20slot%20hours%20by%20site%20and%20role',type:table))"

# wenlong https://fifemon.fnal.gov/kibana/app/kibana#/dashboard/83d7b0c0-8b1c-11ee-804b-5759672b811c?_g=(refreshInterval:(pause:!t,value:0),time:(from:now-1y,mode:quick,to:now))&_a=(description:%27%27,filters:!(),fullScreenMode:!f,options:(darkTheme:!t,hidePanelTitles:!f,useMargins:!t),panels:!((embeddableConfig:(),gridData:(h:14,i:%271%27,w:48,x:0,y:0),id:%2757162130-8b1b-11ee-804b-5759672b811c%27,panelIndex:%271%27,type:visualization,version:%276.8.23%27),(embeddableConfig:(),gridData:(h:16,i:%272%27,w:48,x:0,y:14),id:%275ee81fc0-8b1c-11ee-804b-5759672b811c%27,panelIndex:%272%27,type:visualization,version:%276.8.23%27)),query:(language:lucene,query:%27%27),timeRestore:!t,title:fifebatch-jobs-dune,viewMode:view)

# choose your units

name = 'DUNE monthly slot hours by site and role-2.csv'
inunits="Hr"
HoursPerYear=(24*365)
HoursPerMonth=HoursPerYear/12.
CPUHrPerkHS23=1000/11.
Units = {"MHr":1000000.,"CoreYears":HoursPerYear,"kHS23-Hrs":CPUHrPerkHS23}
Formats = {"Mhr":"10.3f", "CoreYears":"10.1f", "kHS23-Hrs":"10d"}
outunits = "kHS23-Hrs"

# make choices here
lowdate = "2022-12"
highdate = "2023-11" 
units=Units[outunits]
format=Formats[outunits]

In [6]:
Data = {}
ByCountry = {}
sites = []
types = []
dates = []
countries = []

In [7]:
# read in csv and parse into array

with open(name,'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    
    for row in csv_reader:
        #print (row)
        line_count += 1
        if line_count == 1:
            labels = row
            continue
        site = row[0] 
        country = site.split("_")[0]
        date = row[1][0:7] # truncate the day
        type = row[2]
        value = float(row[3].replace(",",""))/units
        

        
        if type not in Data.keys():
            #print ("add type",type)
            Data[type]={}
            types.append(type)
            
        if site not in Data[type].keys():
            #print ("add site",site)
            Data[type][site]={}
        
        
        if type not in types: types.append(type)
        if site not in sites: sites.append(site)
        if date not in dates: dates.append(date)
        if country not in countries: countries.append(country)
        
        Data[type][site][date]=value
         
        
        # print (Data)
print (sites)
print (countries)  
print (dates)

sites.sort()
countries.sort()
 

['US_FermiGrid', 'US_FNAL-FermiGrid', 'NL_SURFsara', 'UK_RAL-Tier1', 'CERN', 'UK_RAL-PPD', 'undefined', 'UK_Manchester', 'CZ_FZU', 'IN_TIFR', 'ES_PIC', 'UK_QMUL', 'NL_NIKHEF', 'UK_Bristol', 'FR_CCIN2P3', 'UK_Lancaster', 'US_BNL', 'UK_Imperial', 'RU_JINR', 'CA_Victoria', 'US_Colorado', 'BR_CBPF', 'UK_Brunel', 'UK_Oxford', 'US_UChicago', 'US_NERSC', 'UK_Edinburgh', 'US_SU-ITS', 'UK_Sheffield', 'UK_Liverpool', 'US_FNAL-T1', 'US_UCSD', 'ES_CIEMAT', 'US_WSU', 'US_Wisconsin', 'US_PuertoRico', 'US_UConn-HPC', 'US_NotreDame', 'CH_UNIBE-LHEP', 'IT_CNAF', 'CA_SFU', 'US_Michigan', 'US_Caltech', 'US_Nebraska', 'US_Omaha', 'US_Swan', 'US_MWT2']
['US', 'NL', 'UK', 'CERN', 'undefined', 'CZ', 'IN', 'ES', 'FR', 'RU', 'CA', 'BR', 'CH', 'IT']
['2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09', '2023-10', '2023-11']


In [8]:
# fill in the blanks 

for type in types:
    for site in sites:
        if site not in Data[type].keys():
            Data[type][site]={}
        for date in dates:
            if date not in Data[type][site].keys():
                Data[type][site][date] = 0.0
        #print (type,site, Data[type][site])
        
# make a NoMARS class

types.append("NoMARS")
Data["NoMARS"] = {}
for site in sites:
    Data["NoMARS"][site] = {}
    for date in dates:
        Data["NoMARS"][site][date] = Data["Total"][site][date]-Data["MARS"][site][date]
        

In [9]:


# trim the date slist
newdates = []
for date in dates:
    if date < lowdate or date > highdate:
                continue
    newdates.append(date)
dates = newdates
print (dates)


['2022-12', '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09', '2023-10', '2023-11']


In [10]:
print ("                                     Usage in %s between %s and %s"%(outunits,lowdate,highdate))
print ("%30s %10s %10s %10s %10s %10s"%("Site","Production","Analysis","NoMARS","MARS","Total"))


totalacrosssite={}
for type in types:
    totalacrosssite[type] = 0.0
for site in sites:
    use = {}
    for type in types:
        use[type] = 0.0        
        for date in dates:
            use[type] += Data[type][site][date]
        if outunits == "Cores": 
            print ("fix core ")
            use[type]/=12. 
        totalacrosssite[type] += use[type]
    
    if "10.3" in format: 
        print ("%30s %10.3f %10.3f %10.3f %10.3f %10.3f"%(site,use["Production"],use["Analysis"],use["NoMARS"],use["MARS"],use["Total"])) 
    else:
        print ("%30s %10d %10d %10d %10d %10d"%(site,use["Production"],use["Analysis"],use["NoMARS"],use["MARS"],use["Total"])) 

        
#totalacrosssite["NoMARS"] = totalacrosssite["Total"] - totalacrosssite["MARS"]
if "10.3" in format:
    print ("%30s %10.3f %10.3f %10.3f %10.3f %10.3f"%("Total",totalacrosssite["Production"],totalacrosssite["Analysis"],totalacrosssite["NoMARS"],totalacrosssite["MARS"],totalacrosssite["Total"]))  
else:
    print ("%30s %10d %10d %10d %10d %10d"%("Total",totalacrosssite["Production"],totalacrosssite["Analysis"],totalacrosssite["NoMARS"],totalacrosssite["MARS"],totalacrosssite["Total"]))      

                                     Usage in kHS23-Hrs between 2022-12 and 2023-11
                          Site Production   Analysis     NoMARS       MARS      Total
                       BR_CBPF       2587        937       3525          0       3525
                        CA_SFU         34         74        109          0        109
                   CA_Victoria        729       1393       2123          0       2123
                          CERN       5476       7482      12958          0      12958
                 CH_UNIBE-LHEP        268         22        290          0        290
                        CZ_FZU       4988       3923       8911          0       8911
                     ES_CIEMAT          1        203        204          0        204
                        ES_PIC       1933       2117       4051          0       4051
                    FR_CCIN2P3       3254       2338       5593          0       5593
                       IN_TIFR        532       3422    

In [11]:
# do by country
for type in types:
    ByCountry[type]={}
    for country in countries:
        ByCountry[type][country]={}
        for date in dates:
            ByCountry[type][country][date] = 0.0
    for site in sites:
        country = site.split("_")[0]
        for date in dates:
            ByCountry[type][country][date]+=Data[type][site][date]

In [12]:
print ("                              Usage in %s between %s and %s"%(outunits,lowdate,highdate))
print ("%30s %10s %10s %10s %10s %10s"%("Country","Production","Analysis","NoMARS","MARS","Total"))


totalacrosssites={}
for type in types:
    totalacrosssite[type] = 0.0
for site in countries:
    use = {}
    for type in types:
        use[type] = 0.0        
        for date in dates:
            use[type] += ByCountry[type][site][date]
        totalacrosssite[type] += use[type]
    #use["NoMARS"] = use["Total"] - use["MARS"]  
    if "10.3" in format:
        print ("%30s %10.3f %10.3f %10.3f %10.3f %10.3f"%(site,use["Production"],use["Analysis"],use["NoMARS"],use["MARS"],use["Total"])) 
    else:
        print ("%30s %10d %10d %10d %10d %10d"%(site,use["Production"],use["Analysis"],use["NoMARS"],use["MARS"],use["Total"])) 

totalacrosssite["NoMARS"] = totalacrosssite["Total"] - totalacrosssite["MARS"]
if "10.3" in format:
    print ("%30s %10.3f %10.3f %10.3f %10.3f %10.3f"%("Total",totalacrosssite["Production"],totalacrosssite["Analysis"],totalacrosssite["NoMARS"],totalacrosssite["MARS"],totalacrosssite["Total"]))  
else:
    print ("%30s %10d %10d %10d %10d %10d"%("Total",totalacrosssite["Production"],totalacrosssite["Analysis"],totalacrosssite["NoMARS"],totalacrosssite["MARS"],totalacrosssite["Total"]))      

                              Usage in kHS23-Hrs between 2022-12 and 2023-11
                       Country Production   Analysis     NoMARS       MARS      Total
                            BR       2587        937       3525          0       3525
                            CA        763       1468       2232          0       2232
                          CERN       5476       7482      12958          0      12958
                            CH        268         22        290          0        290
                            CZ       4988       3923       8911          0       8911
                            ES       1935       2320       4256          0       4256
                            FR       3254       2338       5593          0       5593
                            IN        532       3422       3954          0       3954
                            IT          0        292        292          0        292
                            NL      12557       9420      21977

In [13]:
# Make a table for each type:

In [14]:
# header = "   time in %s by site        "%outunits
# header2 = header

# for date in dates:
#     header += "%10s"%date
#     header2 += ",%s"%date
# header += "     Total"
# header2 += ",Total\n"

# out = {}
# for type in types:
#     outname = "output/%s_BySite_%s_%s_%s.csv"%(type,outunits,lowdate,highdate)
#     out[type] = open(outname,'w')
#     out[type].write(header2+"\n")

# print (sites) 
# for type in types:   
#     for site in sites:
#         result = "%30s"%site
#         outstring = "%s"%site
#         total = 0.0
#         for date in dates:
#             result += " %10.3f"%Data[type][site][date]
#             outstring += ", %10.3f"%(Data[type][site][date])
#             total += Data[type][site][date]
#         #print (outstring)
#         outstring += ",%10.3f\n"%total
#         out[type].write(outstring)
#     out[type].close()

In [15]:
bydate(array=Data,types=types,locations=sites,dates=dates,units=outunits,format=format,tag="BySite")

2022-12 2023-11
['BR_CBPF', 'CA_SFU', 'CA_Victoria', 'CERN', 'CH_UNIBE-LHEP', 'CZ_FZU', 'ES_CIEMAT', 'ES_PIC', 'FR_CCIN2P3', 'IN_TIFR', 'IT_CNAF', 'NL_NIKHEF', 'NL_SURFsara', 'RU_JINR', 'UK_Bristol', 'UK_Brunel', 'UK_Edinburgh', 'UK_Imperial', 'UK_Lancaster', 'UK_Liverpool', 'UK_Manchester', 'UK_Oxford', 'UK_QMUL', 'UK_RAL-PPD', 'UK_RAL-Tier1', 'UK_Sheffield', 'US_BNL', 'US_Caltech', 'US_Colorado', 'US_FNAL-FermiGrid', 'US_FNAL-T1', 'US_FermiGrid', 'US_MWT2', 'US_Michigan', 'US_NERSC', 'US_Nebraska', 'US_NotreDame', 'US_Omaha', 'US_PuertoRico', 'US_SU-ITS', 'US_Swan', 'US_UCSD', 'US_UChicago', 'US_UConn-HPC', 'US_WSU', 'US_Wisconsin', 'undefined']


In [16]:
bydate(array=ByCountry,types=types,locations=countries,dates=dates,units=outunits,format=format,tag="ByCountry")

2022-12 2023-11
['BR', 'CA', 'CERN', 'CH', 'CZ', 'ES', 'FR', 'IN', 'IT', 'NL', 'RU', 'UK', 'US', 'undefined']


In [17]:
# header = "   time in %s by country      "%outunits
# header2 = header

# for date in dates:
#     header += "%10s"%date
#     header2 += ",%s"%date
# header += "     Total"
# header2 += ",Total\n"

# out = {}
# for type in types:
#     outname = "output/%s_ByCountry_%s_%s_%s.csv"%(type,outunits,lowdate,highdate)
#     out[type] = open(outname,'w')
#     out[type].write(header2)

# print (countries) 

# for type in types:  
#     totalbydate = {}
#     for date in dates:
#         totalbydate[date] = 0.0
#     totaltotal = 0.0
#     for site in countries:
#         result = "%30s"%site
#         outstring = "%s"%site
#         total = 0.0
        
#         for date in dates:
#             result += " %10.3f"%(ByCountry[type][site][date])
#             outstring += ", %10.3f"%(ByCountry[type][site][date])
#             total += (ByCountry[type][site][date])
#             totalbydate[date]+= (ByCountry[type][site][date])
#         totaltotal+=total           
#         outstring += ",%10.3f\n"%total
#         #print (outstring)
#         out[type].write(outstring)
#     outstring = "%s"%"Total"
#     for date in dates:
#         outstring += ", %10.3f"%(totalbydate[date])
#     outstring += ",%10.3f\n"%totaltotal 
#     out[type].write(outstring)
#     out[type].close()